In [6]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, fbeta_score, make_scorer, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import shap

In [7]:
# Dataset Básico

df = pd.read_csv("/content/drive/MyDrive/Ps Ligia Time 16/risco_cardiovascular_base.csv")

In [8]:
# Codificação do Target

le = LabelEncoder()
y = le.fit_transform(df["BP_Category"])
X = df.drop("BP_Category", axis=1)

In [9]:
# Divisão dos conjuntos para Treino e Teste (80/20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
# Estruturação do Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)), # Define quanto da variânciaexplicada original será mantida para no treinamento
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

In [11]:
#F2-Score (Atribui peso dobrado ao Recall em relação a Precisão)

f2_scorer = make_scorer(fbeta_score, beta=2, average='weighted')

In [12]:
# Estruturação dos parâmetros para o GridSearch

param_grid = {
    'xgb__n_estimators': [50, 100, 200], # Quantidade de árvores
    'xgb__max_depth': [3, 4, 5],         # Profundidade (baixa para evitar decorar)
    'xgb__learning_rate': [0.01, 0.1],   # Velocidade de aprendizado
    'xgb__gamma': [0, 0.1, 0.2],         # Custo para criar novos nós (poda)
    'pca__n_components': [5, 7, 9]       # Quantos componentes manter
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=f2_scorer, n_jobs=-1)

In [13]:
# GridSearch encontra a configuração do melhor modelo

grid_search.fit(X_train, y_train)

ValueError: 
All the 810 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
162 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 588, in _f

In [ ]:
# Avaliação do modelo

print(f"\n---RELATÓRIO DE CLASSIFICAÇÃO---")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)

# 1. Relatório por classe
print(classification_report(y_test, y_proba, target_names=le.classes_))

# 2. Média de amostragem de todas as classes (AUC-ROC)
auc_score = roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted')
print(f"\nAUC-ROC Global: {auc_score:.4f}")

# 3. F2-Score
f2 = fbeta_score(y_test, y_pred, beta=2, average='weighted')
print(f"\nF2-Score Decisivo: {f2:.4f}")

# 4. Melhores Hiperparâmetros encontrados
print("\nMelhores parâmetros:", grid_search.best_params_)

In [ ]:
# Explicabilidade dos resultados utilizando o SHAP

xgb_model = best_model.named_steps['xgb']
X_test_pca = best_model.named_steps['pca'].transform(best_model.named_steps['scaler'].transform(X_test))

explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test_pca)

# Gráfico do resultado da "força" das features na previsão do modelo
shap.summary_plot(shap_values, X_test_pca)

In [ ]:
# Matriz de Confusão (Visualização da precisão e recall do modelo)

y_pred = grid_search.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap='Blues')
plt.title('Matriz de Confusão: Risco Cardiovascular')
plt.show()